<a href="https://colab.research.google.com/github/angelboat/pytorch_learning/blob/main/_downloads/76d764ad694d0795e494a1edbfb068a6/buildmodel_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # self.flatten是nn.Flatten类的一个实例
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        ) # self.linear_relu_stack是nn.Sequential类的一个实例

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [24]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3])


------------------------------------------------------------------------


In [25]:
print(logits)

tensor([[-0.0441,  0.0232,  0.0639,  0.1123,  0.0130,  0.0031, -0.0176, -0.0829,
          0.0568, -0.0406]], grad_fn=<AddmmBackward0>)


In [28]:
# logits = model(X) 是python"语法糖"（可以理解为一种快捷方式），实际等价于 logits = model.__call__(X)，因为model是nn.Module的子类，
# 继承了父类的__call__方法，该方法调用 forward类方法，所以等价于 logits = model.forward(X)
# 试试call model.forward()的结果
logits_2 = model.__call__(X)
print(logits_2)
logits_3 = model.forward(X)
print(logits_3)

tensor([[-0.0441,  0.0232,  0.0639,  0.1123,  0.0130,  0.0031, -0.0176, -0.0829,
          0.0568, -0.0406]], grad_fn=<AddmmBackward0>)
tensor([[-0.0441,  0.0232,  0.0639,  0.1123,  0.0130,  0.0031, -0.0176, -0.0829,
          0.0568, -0.0406]], grad_fn=<AddmmBackward0>)


In [29]:
X = torch.rand(5, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")
print(pred_probab)

Predicted class: tensor([3, 3, 8, 3, 3])
tensor([[0.0943, 0.1026, 0.1046, 0.1125, 0.0995, 0.0959, 0.0968, 0.0900, 0.1052,
         0.0986],
        [0.0937, 0.1004, 0.1076, 0.1105, 0.0980, 0.0979, 0.1010, 0.0914, 0.1031,
         0.0963],
        [0.0942, 0.1029, 0.1071, 0.1067, 0.0961, 0.0951, 0.1023, 0.0921, 0.1083,
         0.0953],
        [0.0959, 0.0990, 0.1093, 0.1101, 0.0948, 0.0977, 0.1012, 0.0894, 0.1047,
         0.0980],
        [0.0952, 0.1028, 0.1058, 0.1105, 0.0988, 0.0990, 0.0981, 0.0907, 0.1036,
         0.0955]], grad_fn=<SoftmaxBackward0>)


In [16]:
print(pred_probab.sum(dim=1))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [17]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [18]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [19]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [20]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0191,  0.0492, -0.2181, -0.1598,  0.1086,  0.0471,  0.2473, -0.1738,
         -0.2374, -0.4223, -0.2095, -0.3698, -0.2377, -0.0073,  0.1949, -0.5827,
          0.7794,  0.0369,  0.9441, -0.4855],
        [-0.0374, -0.0805, -0.0924,  0.1960,  0.1899,  0.0320,  0.3280, -0.2019,
         -0.1890,  0.1005,  0.0556, -0.0955, -0.3791,  0.1175,  0.3623, -0.4215,
          0.5222, -0.0932,  0.4695, -0.4676],
        [ 0.0818, -0.1337, -0.1184,  0.3375,  0.2981, -0.0368,  0.1412, -0.2371,
         -0.3353, -0.2268,  0.0981, -0.0524, -0.0906,  0.3925,  0.1918, -0.4382,
          0.2123,  0.0346,  0.5867, -0.6009]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0492, 0.0000, 0.0000, 0.1086, 0.0471, 0.2473, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1949, 0.0000, 0.7794, 0.0369,
         0.9441, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.1960, 0.1899, 0.0320, 0.3280, 0.0000, 0.0000,
         0.1005, 0.0556, 0.0000, 0.0000, 0.1175, 0.36

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [22]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0103, -0.0121, -0.0345,  ..., -0.0300,  0.0103,  0.0282],
        [ 0.0212,  0.0332,  0.0202,  ..., -0.0176,  0.0179, -0.0059]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0329, 0.0218], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0074, -0.0246,  0.0300,  ..., -0.0352,  0.0175, -0.0150],
        [ 0.0409,  0.0065, -0.0045,  ..., -0.0129,  0.0148, -0.0203]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
